In [1]:
import pandas as pd

df = pd.read_table('./data/ratings.txt')

df = df.dropna(how='any')
df = df.drop_duplicates(['document'])

df = df[df['document'].apply(lambda it: len(it) >= 10)]
df = df[df['document'].apply(lambda it: len(it) <= 70)]
    
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1


In [2]:
import os

temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'

with open(temp_file, 'w') as f:
    for row in df['document']:
        f.write(str(row) + '\n')

In [3]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix={} --model_type={} --vocab_size={}'.format(
        temp_file, 'retry', 'unigram', 8000)    
)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/aiffel/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp --model_prefix=retry --model_type=unigram --vocab_size=8000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /aiffel/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp
  input_format: 
  model_prefix: retry
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab:

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def sp_tokenize(s, corpus, spm):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open(f'{spm}.vocab', 'r') as f: 
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = pad_sequences(tensor, padding='pre')

    return tensor, word_index, index_word

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, LSTM, Embedding
from keras.models import Sequential

s = spm.SentencePieceProcessor()
s.Load('retry.model')

tensor, word_index, index_word = sp_tokenize(s, df['document'], 'retry')
    
x_train, x_val, y_train, y_val = train_test_split(tensor, df['label'], test_size=0.2)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)

word_vector_dim = 8

model = Sequential()
model.add(Embedding(8000, word_vector_dim))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           64000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                18688     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 82,753
Trainable params: 82,753
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

epochs=20
batch_size=64

history = model.fit(
    x_train, y_train, epochs=epochs, batch_size=batch_size,
    validation_data=(x_val,y_val), callbacks=es, verbose=1)

Epoch 1/20
1597/1597 [==============================] - 42s 6ms/step - loss: 0.4162 - accuracy: 0.8057 - val_loss: 0.3592 - val_accuracy: 0.8419
Epoch 2/20
1597/1597 [==============================] - 9s 5ms/step - loss: 0.3294 - accuracy: 0.8580 - val_loss: 0.3564 - val_accuracy: 0.8463
Epoch 3/20
1597/1597 [==============================] - 9s 5ms/step - loss: 0.3142 - accuracy: 0.8649 - val_loss: 0.3481 - val_accuracy: 0.8478
Epoch 4/20
1597/1597 [==============================] - 9s 6ms/step - loss: 0.2977 - accuracy: 0.8729 - val_loss: 0.3396 - val_accuracy: 0.8500
Epoch 5/20
1597/1597 [==============================] - 9s 6ms/step - loss: 0.2721 - accuracy: 0.8837 - val_loss: 0.3419 - val_accuracy: 0.8522
Epoch 6/20
1597/1597 [==============================] - 9s 6ms/step - loss: 0.2444 - accuracy: 0.8970 - val_loss: 0.3558 - val_accuracy: 0.8486
Epoch 7/20
1597/1597 [==============================] - 9s 5ms/step - loss: 0.2213 - accuracy: 0.9071 - val_loss: 0.3681 - val_accuracy

In [9]:
model.evaluate(x_test, y_test, verbose=0)

[0.42206740379333496, 0.8391402959823608]